In [1]:
import pandas as pd
import numpy as np
import json

data = [json.loads(line) for line in 
        open('/home/left/Desktop/dataMining/set2/ask2/yelp_academic_dataset_business.json', 'r')]

In [2]:
data = [x for x in data if x['city']=='Toronto']
data = np.array(data) # Convert list to numpy array

In [3]:
# Create an array with business with more than 15 reviews
business_col = np.array([])
for i in range(len(data)):
    if data[i]['review_count']>=15:
        business_col = np.append(business_col,data[i]['business_id'])


In [4]:
test = np.array([])
tor_total_business= []

with open('/home/left/Desktop/dataMining/set2/ask2/yelp_academic_dataset_review.json') as reviews_file:
   #business_id = [next(reviews_file) for x in range(1000)]
   #for y in range(100):
        #review_line = next(reviews_file)
        #business_id = json.loads(review_line)
    for line in reviews_file:
        tor_business_reviews = []
        line = json.loads(line)
        mask = np.isin(business_col,line['business_id'])
        mask = business_col[mask]
        if mask.size>0:
            tor_total_business.append(line)

        #tor_business_reviews.append(np.intersect1d(business_col, line['business_id']))
        #if tor_business_reviews[0].size>0:
        #    tor_total_business.append(line)
        
        

In [5]:
len(tor_total_business)

519692

In [6]:
ok = np.array(tor_total_business)

In [7]:
test = []
for i in range(len(tor_total_business)):
    test.append(tor_total_business[i]['user_id'])
test = np.array(test)

In [8]:
unique, counts = np.unique(test, return_counts=True)
ok =dict(zip(unique, counts))
final_l = []
recs=0
for i in range(len(counts)):
    if counts[i]>=15:
        final_l.append(unique[i])
        recs += counts[i]

In [9]:
final_l = np.array(final_l)

In [10]:
final_l.size

5874

# Βημα 2

In [39]:
import pandas as pd 
import math
table_R = pd.read_csv("pruned_data.csv")
print(table_R)

        TZQSUDDcA4ek5gBd6BzcjA  qUWqjjjfpB2-4P3He5rsKw  4.0
0       TZQSUDDcA4ek5gBd6BzcjA  6n_MDeYxU1ihB38be9TkVA  4.0
1       TZQSUDDcA4ek5gBd6BzcjA  jo4KmAqlZ7vxjHIP7IIkAw  5.0
2       TZQSUDDcA4ek5gBd6BzcjA  dsAcgF6qtZy2m6d_yWCrGQ  4.0
3       TZQSUDDcA4ek5gBd6BzcjA  siaRCT2-PkyeXUVKrywcTg  4.0
4       TZQSUDDcA4ek5gBd6BzcjA  SGP1jf6k7spXkgwBlhiUVw  5.0
...                        ...                     ...  ...
197106  3L3CI2Om_2SE3T6hxrJKPQ  j_hB9Gt3VMJAbA2JyvyFjw  4.0
197107  3L3CI2Om_2SE3T6hxrJKPQ  WKcVegcUjAld1S737a_QHw  4.0
197108  3L3CI2Om_2SE3T6hxrJKPQ  1FL3oE2mqq_EFAYPd1TWUg  5.0
197109  3L3CI2Om_2SE3T6hxrJKPQ  hlrZHM4D48XiQtXh6cRg_w  1.0
197110  3L3CI2Om_2SE3T6hxrJKPQ  _cVCzKQGt23KKW1M07Yvkw  5.0

[197111 rows x 3 columns]


In [40]:
print("Sample data should have {} rows \n".format(math.ceil(len(table_R)*0.05)))
sample_data = table_R.sample(frac=0.05)
print(len(sample_data))

Sample data should have 9856 rows 

9856


In [41]:
print("After we remove the samples R should have {}".format(orig_table_R_len - len(sample_data)))
table_R = table_R.drop(sample_data.index)
print(len(table_R))

After we remove the samples R should have 187255
187255


In [18]:
bs = []
for x in range(len(data)):
    if data[x]['review_count']>=15:
        bs.append(data[x])
print(len(bs))

7602
